In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
import cv2
import numpy as np
from os import listdir as ld
from os.path import join as pj
import pandas as pd
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.utils.data as data

# Dataset
from dataset.detection.dataset import insects_dataset_from_voc_style_txt, collate_fn
# Loader
from IO.loader import load_path, load_images, load_annotations_path, load_annotations, get_anno_recs
# Predict
from model.refinedet.utils.predict import test_prediction_with_cls
# utils
from IO.utils import refine_result_by_ovthresh_with_cls, result_formatter_with_cls
# Evaluate
from evaluation.det2cls.evaluate import create_imwise_gt, get_cls_accuracy_per_class_with_cls
from evaluation.det2cls.visualize import vis_detections

# Test Config

In [ ]:
class args:
    # experiment name
    experiment_name = "crop_b2_2_4_8_16_32_im512_cls"
    # paths
    data_root = "/home/tanida/workspace/Insect_Phenology_Detector/data"
    test_image_root = "/home/tanida/workspace/Insect_Phenology_Detector/data/test_refined_images"
    model_root = pj("/home/tanida/workspace/Insect_Phenology_Detector/output_model/detection/RefineDet", experiment_name)
    figure_root = pj("/home/tanida/workspace/Insect_Phenology_Detector/figure/det2cls/RefineDet_only", "crop_b2_2_4_8_16_32_im512_cls")
    test_anno_folders = ["annotations_4"]
    # training config
    input_size = 512 # choices=[320, 512, 1024]
    crop_num = (5, 5)
    tcb_layer_num = 5
    rm_last = True

# Model Config

In [ ]:
if args.tcb_layer_num == 4 and args.rm_last == False:
    from model.refinedet.config import tcb_4_rm_false as insect_refinedet
elif args.tcb_layer_num == 4 and args.rm_last == True:
    from model.refinedet.config import tcb_4_rm_true as insect_refinedet
elif args.tcb_layer_num == 5 and args.rm_last == False:
    from model.refinedet.config import tcb_5_rm_false as insect_refinedet
elif args.tcb_layer_num == 5 and args.rm_last == True:
    from model.refinedet.config import tcb_5_rm_true as insect_refinedet
elif args.tcb_layer_num == 6 and args.rm_last == False:
    from model.refinedet.config import tcb_6_rm_false as insect_refinedet

# Set cuda

In [ ]:
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')

# Model

In [ ]:
if args.rm_last == True:
    from model.refinedet.refinedet_rmlast import build_refinedet
else:
    from model.refinedet.refinedet import build_refinedet

### Make data

In [ ]:
print('Loading dataset for test ...')
test_dataset = insects_dataset_from_voc_style_txt(args.test_image_root, args.input_size, args.crop_num, "RefineDet", training=False)
test_data_loader = data.DataLoader(test_dataset, 1, num_workers=1, shuffle=False, collate_fn=collate_fn)
print('Loading annotation for test...')
annos, imgs = load_path(args.data_root, "refined_images", args.test_anno_folders)
images = load_images(imgs)
annotations_path = load_annotations_path(annos, images)
anno = load_annotations(annotations_path)
imagenames, recs = get_anno_recs(anno, each_flag=True)

### Get model config

In [ ]:
cfg = insect_refinedet[str(args.input_size)]

### Load model

In [ ]:
print("Loading model for test ...")
model = build_refinedet('test', insect_refinedet, args.input_size, args.tcb_layer_num, num_classes=13).cuda()
load_name = pj(args.model_root, 'RefineDet{}_{}.pth'.format(args.input_size, "final"))
model.load_state_dict(torch.load(load_name))

In [ ]:
model

# --- result analysis ---

In [ ]:
result = test_prediction_with_cls(model, test_data_loader, args.crop_num, 13, nms_thresh=0.3)
result = refine_result_by_ovthresh_with_cls(result, 13, ovthresh=0.3)
cls_lbl_dic = {1:6, 2:0, 3:1, 4:2, 5:6, 6:3, 7:4, 8:5, 9:6, 10:6, 11:6, 12:6}
result = result_formatter_with_cls(result, cls_lbl_dic)

In [ ]:
gt_dict, npos = create_imwise_gt(recs)

In [ ]:
# arrange index to label
# not considered label = -1
name2lbl = {
    'Coleoptera': 6, 
    'Diptera': 0, 
    'Ephemeridae': 1, 
    'Ephemeroptera': 2, 
    'Hemiptera': 6, 
    'Lepidoptera': 3, 
    'Plecoptera': 4, 
    'Trichoptera': 5, 
    'medium insect': 6, 
    'small insect': 6, 
    'snail': 6, 
    'spider': 6
}

In [ ]:
accs, recalls, precisions = get_cls_accuracy_per_class_with_cls(result, gt_dict, name2lbl, add_divide_model=False, n_class_when_not_use_divide_model=7)

In [ ]:
np.mean(accs)

In [ ]:
np.mean(recalls)

In [ ]:
np.mean(precisions)

In [ ]:
accs

In [ ]:
recalls

In [ ]:
precisions

### --- compare ground truth and output ---

In [ ]:
add_divide_model = False
n_class_when_not_use_divide_model = 7

In [ ]:
im_index = 0
if os.path.exists(args.figure_root) is False:
    os.makedirs(args.figure_root)

In [ ]:
colors = ["white", "red", "lime", "blue", "yellow", "fuchsia", "aqua", "gray", "maroon", "green", "navy", "olive", "purple", "teal"]
insect_name =  ['Diptera', 'Ephemeridae', 'Ephemeroptera', 'Lepidoptera', 'Plecoptera', 'Trichoptera', 'Other_insects']
# arrange index to label
# not considered label = -1
# to see ground truth, use medium and small label
name2lbl = {
    'Coleoptera': 6, 
    'Diptera': 0, 
    'Ephemeridae': 1, 
    'Ephemeroptera': 2, 
    'Hemiptera': 6, 
    'Lepidoptera': 3, 
    'Plecoptera': 4, 
    'Trichoptera': 5, 
    'medium insect': 6, 
    'small insect': 6, 
    'snail': 6, 
    'spider': 6
}

In [ ]:
img = np.asarray(Image.open(pj("/home/tanida/workspace/Insect_Phenology_Detector/data/refined_images", imagenames[im_index]+".png")))
out = result[imagenames[im_index]]["output_lbl"]
if add_divide_model is True:
    out = np.asarray([output for i, output in enumerate(out) if result[imagenames[im_index]]["divide_lbl"][i] == 0])
    result_coord = np.asarray([coord for i, coord in enumerate(result[imagenames[im_index]]['coord']) if result[imagenames[im_index]]["divide_lbl"][i] == 0])
else:
    out_mask = out == (n_class_when_not_use_divide_model - 1)
    out = np.asarray([output for i, output in enumerate(out) if out_mask[i] == False])
    result_coord = np.asarray([coord for i, coord in enumerate(result[imagenames[im_index]]['coord']) if out_mask[i] == False])

for i in range(len(insect_name)):
    lbl_filter = out == i
    result_coord_filtered_by_lbl = result_coord[lbl_filter]
    gt_lbls = np.asarray([name2lbl[name] for name in gt_dict[imagenames[im_index]]['default_name']])
    lbl_filter = gt_lbls == i
    gt_coord_filtered_by_lbl = gt_dict[imagenames[im_index]]['bbox'][lbl_filter]
    
    img = vis_detections(img, result_coord_filtered_by_lbl, class_name=insect_name[i], color_name=colors[i])
    img = vis_detections(img, gt_coord_filtered_by_lbl, class_name=insect_name[i], color_name=colors[i])

img = Image.fromarray(img)
img.save(pj(args.figure_root, imagenames[im_index]+".png"))
print(imagenames[im_index])
im_index += 1